# Manejo de Permisos, Workflows y Jobs en entornos productivos

En los entornos de datos modernos, como Databricks, la correcta gestión de permisos, la automatización de procesos mediante workflows y la orquestación de jobs son pilares fundamentales para garantizar la seguridad, la eficiencia y la trazabilidad de las operaciones. 

En este notebook aprenderás los conceptos clave y mejores prácticas para:
- Gestionar el acceso seguro a los datos y recursos.
- Automatizar tareas repetitivas y procesos complejos.
- Orquestar flujos de trabajo robustos y auditables.

Esto es especialmente relevante en entornos productivos, donde la gobernanza y la escalabilidad son críticas para el éxito de los proyectos de analítica y ciencia de datos.

## 1. Unity Catalog

Unity Catalog es la solución de gobernanza de datos de Databricks que permite gestionar de forma centralizada el acceso, la organización y la auditoría de todos los activos de datos de una organización. Facilita la implementación de políticas de seguridad y cumplimiento normativo, permitiendo un control granular sobre quién puede ver o modificar cada recurso.

Con Unity Catalog, las empresas pueden:
- Unificar la gestión de permisos en todos los workspaces y entornos.
- Garantizar la trazabilidad y el linaje de los datos.
- Cumplir con regulaciones de privacidad y seguridad de la información.

### Objetivos
- Comprender la arquitectura y los componentes de Unity Catalog.
- Implementar modelos de seguridad granulares para proteger los datos sensibles.
- Gestionar permisos a nivel de objeto (catálogo, esquema, tabla).
- Utilizar el lineage para auditar y rastrear el flujo de los datos a lo largo de los procesos.

Unity Catalog proporciona una capa de gobierno centralizada que simplifica la administración de datos en la organización. Sus principales ventajas incluyen:
- Un modelo unificado de gobierno para todos los datos, independientemente del motor o lenguaje utilizado.
- Control de acceso fino: puedes definir permisos a nivel de catálogo, esquema, tabla, vista.
- Lineage completo: permite rastrear el origen y las transformaciones de los datos, facilitando auditorías y análisis de impacto.
- Auditoría centralizada: todas las acciones quedan registradas para su revisión.

**Estructura jerárquica de Unity Catalog:**
1. **Metastore** (nivel más alto): almacén central de metadatos.
2. **Catálogos**: agrupan esquemas y objetos, suelen representar entornos (dev, staging, prod).
3. **Esquemas**: equivalentes a bases de datos, agrupan tablas y vistas.
4. **Tablas, vistas y funciones**: objetos de datos y lógica.

### Estructuras de Unity Catalog

Las estructuras jerárquicas de Unity Catalog permiten organizar los datos de forma lógica y segura. Esta organización facilita la asignación de permisos y la segmentación de los datos según necesidades de negocio o cumplimiento.

Vamos a crear un catálogo, que será el contenedor principal de los datos y recursos que utilizaremos durante esta sesión. Esto nos permitirá trabajar de forma aislada y controlada.

In [0]:
dbutils.widgets.text("catalogo", "sesion_2", "Nombre del Catálogo que vamos a usar durante esta sesion")
catalog_name = dbutils.widgets.get("catalogo")

In [0]:
# Crear catálogo
spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_name}")
sql(f"USE CATALOG {catalog_name}")

Ahora crearemos un esquema dentro del catálogo, que funcionará como una base de datos lógica para organizar nuestras tablas y vistas de la sesión.

In [0]:
dbutils.widgets.text("esquema", "retail_ventas", "Nombre del Esquema")
schema_name = dbutils.widgets.get("esquema")

In [0]:
sql(f"CREATE SCHEMA IF NOT EXISTS {schema_name}")

#### Usuarios y Grupos

La gestión de usuarios y grupos es fundamental para aplicar políticas de seguridad y control de acceso en Databricks. Los grupos permiten asignar permisos de forma eficiente y escalable, evitando la gestión individualizada usuario por usuario.

Ahora vamos a otorgar permisos a los usuarios. Para ello, primero crearemos grupos, ya que la asignación de permisos a grupos es más eficiente y segura que hacerlo a usuarios individuales. Por defecto, solemos trabajar con el usuario admin dentro del grupo admins, pero en entornos reales es recomendable definir grupos específicos para cada rol o equipo.

In [0]:
display(sql("SHOW GROUPS"))

Los usuarios y grupos pueden crearse tanto mediante comandos SQL como utilizando la interfaz de administración de Databricks. Esto permite automatizar la gestión de identidades y facilitar la integración con sistemas externos.

In [0]:
%sql
CREATE GROUP data_engineers_b;


In [0]:
sql("CREATE GROUP `data_analysts_b`")

In [0]:
%sql
SHOW GRANTS ON CATALOG `sesion_2`;

Se pueden asignar permisos a diferentes niveles:
- Catálogos
- Esquemas
- Tablas

Los permisos se otorgan a entidades denominadas *principals*, que pueden ser:
- **Usuarios**: identificados por su nombre o correo electrónico.
- **Service principals**: identidades técnicas asociadas a aplicaciones o servicios automatizados.
- **Grupos**: conjuntos de usuarios con permisos comunes.

Esta flexibilidad permite implementar modelos de seguridad adaptados a las necesidades de cada organización.

In [0]:
display(sql("SHOW GROUPS"))

Vamos a intentar asignar un permiso a uno de los grupos que hemos creado anteriormente. Observa que, si el grupo no está correctamente sincronizado a nivel de cuenta, se producirá un error. Esto ilustra la importancia de gestionar los grupos en el lugar adecuado.

In [0]:
sql(f"GRANT SELECT ON SCHEMA {schema_name} TO `data_analysts_b`")

**Importante**: los grupos creados únicamente en el workspace no son reconocidos por Unity Catalog. Lo mismo ocurre con los usuarios.
- Unity Catalog solo reconoce grupos y usuarios definidos a nivel de cuenta (creados manualmente o sincronizados desde sistemas de identidad como Azure AD, Okta, etc.).
- Si un grupo como `data_analysts` existe solo en el workspace, Unity Catalog no lo podrá utilizar como principal para asignar permisos.

Por eso, es fundamental gestionar los grupos a nivel de cuenta para aprovechar todas las capacidades de seguridad de Unity Catalog.

Para saber si un grupo es de cuenta o solo del workspace, existen varias formas de comprobarlo:
1. Usando la Account Console de Databricks (https://accounts.cloud.databricks.com).
2. Revisando la configuración de administración (Admin settings) en la esquina superior derecha de la interfaz.

Esto es importante para asegurarse de que los grupos utilizados en las políticas de seguridad sean válidos para Unity Catalog.

#### Utilización Usuarios de Cuenta

Para crear un grupo a nivel de cuenta existen dos métodos principales:
- **Account Console**: método gráfico y sencillo, recomendado para la mayoría de los casos.
- **Databricks Account API**: método avanzado y automatizable, ideal para integraciones y grandes organizaciones. Permite crear grupos mediante scripts y pipelines de automatización.

A continuación, se muestra un ejemplo de cómo crear un grupo usando la API, lo que resulta útil para procesos repetitivos o integraciones con sistemas externos.
  ```bash
  curl -X POST https://accounts.cloud.databricks.com/api/2.0/accounts/<ACCOUNT_ID>/groups \
  -H "Authorization: Bearer <ACCOUNT_TOKEN>" \
  -H "Content-Type: application/json" \
  -d '{"display_name": "data_engineers"}'
  ```

  También tenéis un archivo en el repositorio llamado `group_creator.sh` para automatizar este proceso

Vamos a repasar el proceso para crear y sincronizar grupos a nivel de cuenta mediante la consola (que es lo más común):
1. Accede a la Account Console y crea los grupos `data_engineers` y `data_analysts`.
2. Vuelve a la página de configuración del workspace y comprueba que los grupos aún no aparecen.
3. Otorga permisos al workspace para acceder a los grupos de cuenta.
4. Vuelve a consultar los grupos desde el workspace y verifica que ya están disponibles.

Este flujo garantiza que los grupos estén correctamente sincronizados y puedan ser utilizados en las políticas de Unity Catalog.

Ahora que ya tenemos los grupos sincronizados nos falta asignar pemisos para poder utilizar los grupos apropiadamente

In [0]:
%sql
SHOW GRANTS on catalog sesion_2;

In [0]:
%sql
GRANT ALL PRIVILEGES ON CATALOG sesion_2 TO `data_engineers`;


Aquí tienes el enlace a la documentación oficial de Databricks sobre los permisos disponibles. Es importante consultar esta referencia, ya que los privilegios varían según el tipo de entidad (catálogo, esquema, tabla, etc.) y se actualizan con frecuencia:
[Privilegios](https://docs.databricks.com/aws/en/data-governance/unity-catalog/manage-privileges/privileges)

Revisar la documentación te ayudará a aplicar las mejores prácticas y evitar errores de configuración.

Los permisos en Unity Catalog se heredan de forma jerárquica. Es decir, si otorgas un permiso a un catálogo, ese permiso se aplicará también a los esquemas y tablas que contiene, salvo que se sobrescriba explícitamente en un nivel inferior.

Por ejemplo, si un grupo tiene acceso de lectura a un catálogo, podrá leer todas las tablas y vistas de los esquemas dentro de ese catálogo, a menos que se restrinja en algún esquema o tabla específica.

El siguiente gráfico ilustra la jerarquía de permisos en Unity Catalog. Observa cómo los permisos fluyen desde el nivel superior (catálogo) hacia los niveles inferiores (esquema, tabla, columna), permitiendo una gestión flexible y escalable de la seguridad de los datos.

![Gráfico de jerarquía de permisos](https://docs.databricks.com/aws/en/assets/images/object-hierarchy-199f1519005d35d39b82212b8b71a079.png)

In [0]:
%sql
SHOW GRANTS ON SCHEMA sesion_2.retail_ventas;


Al igual que los permisos se otorgan también se pueden revocar mediante este comando

In [0]:
sql(f"REVOKE ALL PRIVILEGES ON CATALOG sesion_2 FROM `data_engineers`")

In [0]:
%sql
SHOW GRANTS on catalog sesion_2;

#### Comparativa entre Usuarios y Grupos en Workspace vs Cuenta

La siguiente tabla resume las diferencias clave entre los usuarios y grupos gestionados a nivel de workspace y los gestionados a nivel de cuenta. Es fundamental entender estas diferencias para diseñar una estrategia de seguridad robusta y compatible con Unity Catalog.

| Característica                               | Workspace User/Group     | Account User/Group      |
|----------------------------------------------|--------------------------|-------------------------|
| Visibles solo en un workspace                | ✅ Sí                    | ❌ No (son globales)   |
| Útiles para permisos en notebooks, jobs, etc.| ✅ Sí                    | ✅ Sí                  |
| Se usan para Unity Catalog (datos)           | ❌ No                    | ✅ Sí                  |
| Gobernanza centralizada de datos             | ❌ No                    | ✅ Sí                  |
| Accesibles desde todos los workspaces        | ❌ No                    | ✅ Sí                  |
| Gestión desde Workspace UI                   | ✅ Sí                    | ❌ No (solo desde Account Console o SCIM API) |
| Sincronizables desde IdP (Azure AD, Okta)    | ❌ No                    | ✅ Sí                  |
| Ideal para producción                        | ❌ No                    | ✅ Sí                  |

Como regla general, para entornos productivos y para aprovechar todas las capacidades de Unity Catalog, utiliza siempre usuarios y grupos de cuenta.



## Ejercicios


En este ejercicio vamos a:

1. Crear un esquema y una tabla sensible en Unity Catalog
2. Crear grupos y usuarios
3. Asignar permisos diferenciados a distintos perfiles
4. Validar los accesos mediante consultas

### Parte A – Creación del esquema y la tabla

Vamos a crear un esquema `retail` dentro del catálogo `empresa_datos`, y una tabla que contiene datos estas columnas:
- id_ventas
- fecha
- producto_id
- cantidad
- precio
- email_cliente (es un dato sensible, ya veremos luego porqué decimos esto)
- total (cantidad*precio)

In [0]:
%sql
-- Parámetros (asegúrate de tener estos grupos creados a nivel de cuenta)
CREATE SCHEMA IF NOT EXISTS sesion_2.retail;

USE CATALOG sesion_2;
USE SCHEMA retail;

CREATE OR REPLACE TABLE ventas_clientes (
  id_venta STRING,
  fecha DATE,
  producto_id STRING,
  cantidad INT,
  precio DOUBLE,
  email_cliente STRING,
  total DOUBLE GENERATED ALWAYS AS (cantidad * precio)
)
COMMENT 'Tabla con datos sensibles de clientes y ventas';


###  Parte B – Configuración de permisos a grupos

Crea los 3 siguientes grupos de usuarios
- `data_engineers`: acceso total (lectura, escritura)
- `data_analysts`: solo lectura general
- `data_scientists`: acceso solo a columnas no sensibles

Asegúrate de que estos grupos existen en **Account Console** y están sincronizados con tu workspace.


In [0]:
%sql
-- Asignar permisos a data_engineers
GRANT ALL PRIVILEGES ON CATALOG sesion_2 TO `data_engineers`;
GRANT ALL PRIVILEGES ON SCHEMA retail TO `data_engineers`;
GRANT ALL PRIVILEGES ON TABLE ventas_clientes TO `data_engineers`;

In [0]:
%sql
-- Verifica los permisos sobre el catálogo
SHOW GRANTS ON CATALOG sesion_2;


In [0]:
%sql
-- Permisos limitados para data_analysts
GRANT USE CATALOG ON CATALOG sesion_2 TO `data_analysts`;
GRANT USE SCHEMA ON SCHEMA retail TO `data_analysts`;
GRANT SELECT ON TABLE ventas_clientes TO `data_analysts`;

In [0]:
%sql
-- Verifica los permisos sobre el esquema
SHOW GRANTS ON SCHEMA sesion_2.retail;


In [0]:
%sql
-- Permisos limitados para data_scientists (solo columnas específicas)
GRANT USE CATALOG ON CATALOG sesion_2 TO `data_scientists`;
GRANT USE SCHEMA ON SCHEMA retail TO `data_scientists`;

CREATE VIEW sesion_2.retail.ventas_clientes_restricted AS
SELECT ID_VENTA, fecha, producto_id, cantidad, precio, total 
FROM sesion_2.retail.ventas_clientes;

GRANT SELECT ON VIEW sesion_2.retail.ventas_clientes_restricted TO `data_scientists`;

In [0]:
%sql
-- Verifica los permisos sobre la tabla
SHOW GRANTS ON TABLE sesion_2.retail.ventas_clientes_restricted;



### Parte C – Verificación de permisos asignados a grupos

A continuación, vamos a comprobar que los grupos `data_engineers`, `data_analysts` y `data_scientists` tienen los permisos esperados sobre el catálogo, esquema y tabla.


## 2. Job y Workflows

###  Introducción a los Jobs

Un **Job** en Databricks es una forma de automatizar la ejecución de notebooks, scripts o JARs, ya sea de forma programada o por eventos. Cada Job puede contener múltiples **tareas** (tasks) que se organizan como un **Workflow** con dependencias entre ellas.

Ejemplos de uso:
- Cargar datos a diario desde un sistema fuente.
- Transformar datos mediante una secuencia de notebooks.
- Ejecutar pipelines de ML y enviar alertas si fallan.

En esta sección, exploraremos cómo crear un Job con múltiples tareas conectadas.


Vamos a ver un ejemplo:
Vamos a crear un job que cargue un fichero de un volumen y lo guarde en una delta table apendeando los datos. Antes de esto vamos a hacer lo siguiente:
 - Descargarnos el fochero `datos_clientes.csv` y subirlo a nuestro volumen
 - Cuando lo vayamos a ejucutar hacerlo con estos parametros:
 ```bash
 ["--input_volume=/Volumes/mi_catalogo/mi_esquema/mi_volumen",
 "--input_file=datos_*.csv",
 "--output_table=mi_catalogo.mi_esquema.tabla_clientes"]
 ```

Ahora consultamos la tabla para ver que ha ido bien la cosa:

In [0]:
%sql
SELECT * FROM sesion_2.retail_ventas.tabla_clientes;

### Ejercicio Job

Tu tarea es crear un proceso parametrizado que lea datos y los guarde en una tabla Delta. Para ello hay que hacer lo siguiente:
1. Crear un fichero `.py` con la lógica de la ejcución (utilizando la plantilla que se llama `ejercicio_job`) y haga lo siguiente:
    - Lea el archivo JSON `datos_ejercicios.json` desde un Volumen.
    - Valide que los datos tengan las columnas correctas.
    - Añada metadatos (fecha_procesamiento y fuente y nombre).
    - Filtrar los pedidos con `total_aamount <=0`
    - Cargue los datos en una tabla Delta (creándola si no existe).
2. Cree el job con las siguientes características:
    - Poner parámetros de entrada
    - Ejecución automática cada 5 minutos

###  Introducción a los Workflows
- Orquestación de múltiples tareas dentro de un Job.
- Similar a DAGs en Airflow: define dependencias, triggers, condiciones, etc.
- Compatible con notebooks, Python scripts, SQL y tareas externas.

Vamos a ver ahora cómo crear un Workflow con 5 fases:
  1. ingestar_datos
  2. validar_datos
  3. castear_datos
  4. agrupar_datos
  5. crear_alertas

#### Lineage

Antes de pasar a los ejercicios vamos a ver el **lineage** de una Delta Table. Es el rastro de datos que muestra el origen, las transformaciones y las dependencias de una tabla dentro de un entorno de Databricks, incluyendo:
- Qué datos se usaron como fuente (archivos, otras tablas, bases de datos, etc.)
- Qué transformaciones se aplicaron (joins, filtros, agregaciones, etc.)
- Qué notebooks, workflows o jobs intervinieron
- Quién ejecutó esas transformaciones y cuándo

Para verlo se puede hacer directamente en la interfaz web:
1. Ve al Catalog.
2. Selecciona un catálogo y esquema.
3. Haz clic en una tabla.
4. Ve a la pestaña Lineage.

## Ejercicios

Crear un Workflow con 4 tareas:
1. Cargar los datos de vuelos y de aeropuertos desde 2 archivos CSV a 2 tabla Delta (primero vamos a subir a mano los archivos desde el repo a un Volumen).
2. Procesar los datos de vuelos a otra tabla delta añadiendo 2 columnas: el tiempo de vuelo y el retraso.
3. Unir ambas tablas para crear una tercera con los vuelos y sus nombres de aeropuertos.

*Pistas*:
  1. Para desarrollar las tareas de ingestas vale el mismo archivo. Únicamente hay que crear un proceso que lea un fichero csv desde una ruta parametrizable y lo cargue en una tabla delta
  2. Se puede asignar a diferentes `Task` un mismo fichero `.py` y cambiar los argumentos de entrada del programa

## PipeLines y Delta Live Tables (Avanzado) 

Las Delta Live Tables (DLT) en Databricks son una funcionalidad que permite crear, gestionar y mantener pipelines de datos de forma declarativa y automatizada y se utilizan en el apartado Workflow -> Pilines. Están diseñadas para simplificar la ingestión, transformación y calidad de los datos dentro del lakehouse de Databricks, aprovechando el formato Delta Lake.

### Características principales

- Declarativo: defines las transformaciones como funciones o vistas, sin preocuparte de la ejecución secuencial.
- Automatizado: Databricks determina el orden de ejecución, gestiona los cambios y optimiza el rendimiento.
- Incremental: trabaja eficientemente con datos nuevos sin tener que reprocesar todo el conjunto.
- Data Quality con Expectations: puedes definir condiciones para validar los datos (ej. “no debe haber valores nulos en una columna clave”).

#### Ejemplo de un procesamiento incremental de datos

- Ventaja: DLT detecta automáticamente nuevos datos y procesa solo lo necesario.


```python
@dlt.table(
    comment="Silver layer: cleaned user data",
    partition_cols=["date"],
    table_properties={"quality": "silver"}
)
def users_clean():
    return (
        dlt.read_stream("users_bronze")
        .filter("is_active = true")
    )
  ```

```python
# En un notebook tradicional
df = spark.read.format("delta").load("/mnt/bronze/users")
df = df.filter("is_active = true")
df.write.format("delta").mode("append").partitionBy("date").save("/mnt/silver/users")

# Requiere gestión manual de:
# - Watermarks (para streaming)
# - Control de particiones
# - Estado del último procesamiento
```